# COCO Image Tiler

Tile larger COCO annotated images into smaller COCO annotated images

This involves cutting up the images and fixing the COCO annotations so that they align with the new images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import io
import PIL
from PIL import Image


import os
print(f"Current OS: {os.name}") 

## Utils

In [ ]:
# Plot an image from a TF Record with its bounding boxes and labels

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plt_image_with_labels(image, annotations, categories):
    ## Display the image, with bboxes and labels
    # Create figure and axes
    fig, ax = plt.subplots(figsize=(7, 7))

    # Display the image
    ax.imshow(image)
    height = annotations["image/height"]
    width = annotations["image/width"]
    # Create Rectangle patches, i.e. bbox add rectangles to the image
    for j in range(len(annotations['image/object/class/label'])):
        x1 = annotations["image/object/bbox/xmin"][j] * width
        y1 = annotations["image/object/bbox/ymin"][j] * height
        x2 = annotations["image/object/bbox/xmax"][j] * width - x1
        y2 = annotations["image/object/bbox/ymax"][j] * height - y1
        
        category_num = annotations["image/object/class/label"][j]-1
        label = categories[category_num]["name"]
        rect = patches.Rectangle((x1, y1), x2, y2, linewidth=1, edgecolor='red', facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
        ax.text(x1,y1, label, horizontalalignment='left', fontsize=8, color='red')

    plt.show()

In [ ]:
# Plot an image from a TF Record with its bounding boxes and labels

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plt_image_with_labels2(image, annotations, categories):
    ## Display the image, with bboxes and labels
    # Create figure and axes
    fig, ax = plt.subplots(figsize=(7, 7))

    # Display the image
    ax.imshow(image)
    height = annotations['image']['height']
    width = annotations['image']['width']
    # Create Rectangle patches, i.e. bbox add rectangles to the image
    for row in annotations['annotations']:
        x1, y1, x2, y2 = row['bbox']
        
        category_num = row['category_id']
        label = categories[category_num]
        rect = patches.Rectangle((x1, y1), x2, y2, linewidth=1, edgecolor='red', facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
        ax.text(x1,y1, label, horizontalalignment='left', fontsize=8, color='red')

    plt.show()

In [ ]:
def counter(start = None):
    if start == None:
        count = 0
    else:
        count = start
    def increment(cmd = None):
        nonlocal count
        if cmd == 'READ':
            #no imcreament, just read the count
            cmd = None
        else:
            if cmd == None:
                count += 1
            else:
                count = cmd
                cmd = None
        return count
    return increment

In [ ]:
def filename_append(filename, append_text):
    old_file = os.path.splitext(filename)
    new_file_name = old_file[0]+'_'+str(append_text)+old_file[1]
    return new_file_name

### Get COCO file and prep COCO Info

In [ ]:
# Define a list of folder paths to be created (if needed) and used later
root_dir = os.path.join('E:', os.sep ,'Users', 'Vince', 'Datasets', 'NZRC', 'Gordon', 'CycloneGitaRawImagery')

paths = {
    'ROOT_PATH' : root_dir,
    'IMAGE_SRC_PATH' : os.path.join(root_dir,'ML4DR_20210910_01'),
    'IMAGE_DST_PATH' : os.path.join(root_dir, 'ML4DR_20210910_01', 'tiles'),
    'ANNOTATION_PATH' : os.path.join(root_dir, 'ML4DR_20210910_01')
}

files = {
    'SRC_ANNOTATIONS_FILE' : os.path.join(paths['ANNOTATION_PATH'], 'ML4DR_20210910_01_coco.json'),
    'DST_ANNOTATIONS_FILE' : os.path.join(paths['ANNOTATION_PATH'], 'tiles', 'ML4DR_20210910_01_coco_tiled.json')
}

In [ ]:
#Load COCO file information

import json

def load_coco_info(annotation_file):
    coco_info = {}
    #Load file info from COCO file
    with open(annotation_file, "r") as f:
        coco_info['info'] = json.load(f)["info"]

    #Load image refs from COCO file
    with open(annotation_file, "r") as f:
        coco_info['images'] = json.load(f)["images"]

    #Load annotations from COCO file
    with open(annotation_file, "r") as f:
        coco_info['annotations'] = json.load(f)["annotations"]

    #Load licenses from COCO file
    with open(annotation_file, "r") as f:
        coco_info['licenses'] = json.load(f)["licenses"]

    #Load categories from COCO file
    with open(annotation_file, "r") as f:
        coco_info['categories'] = json.load(f)["categories"]

    return coco_info

In [ ]:
#Rearrange COCO file info into a form that can be used to create the TF Record file
# this requires the annotation info for each image be associated to the image info
# and it requires a category dictionary to convert from category_id to category_text

#Work through coco-images
# Extract the image attributes
# For each image get a list of annotations
# Add the annotations list to the image

coco_info = load_coco_info(files['SRC_ANNOTATIONS_FILE'])

imageInfoList = []
for coco_image_info in coco_info['images']:
    annotationsList = []
    for annotation in coco_info['annotations']:
        if coco_image_info['id'] == annotation['image_id']:
            annotationsList.append(annotation)
    if len(annotationsList)>0:
        image_info={}
        image_info['image'] = coco_image_info
        image_info['annotations'] = annotationsList
        imageInfoList.append(image_info)
#return imageInfoList

categoriesDict = {}
labels = []
for cat in coco_info['categories']:
    labels.append({'name':cat['name'], 'id':cat['id']})
    categoriesDict[cat['id']] = cat['name']
#return categoriesDict

### Process COCO info

In [ ]:
# Extract bboxes that are within the crop_box
# and adjust their coordinates to fit within the cropped image

def get_bboxes(crop_box, image_info, image_id, annotation_counter):
    adj_annotations = []
    for a in image_info['annotations']: #Iterate through the annotations associated with the image
        bbox_x1, bbox_y1, bbox_x2, bbox_y2 = a['bbox']
        bbox_x2 += bbox_x1
        bbox_y2 += bbox_y1
        crop_x1, crop_y1, crop_x2, crop_y2 = crop_box
        
        b = {} # create a new dict to copy the values into
        
        # If top-left bbox corner is inside the crop box
        if crop_x1 < bbox_x1 and crop_y1 < bbox_y1:
            # If bottom-right bbox corner is inside the crop box
            if bbox_x2 < crop_x2 and bbox_y2 < crop_y2:
                # the bbox is inside the copy box
                #print('       info =', a['bbox'])
                #print('   crop_box =', crop_x1, crop_y1, crop_x2, crop_y2)
                #print('       bbox =', bbox_x1, bbox_y1, bbox_x2, bbox_y2)
                # adjust the bbox coords to be inside the cropped image
                adj_bbox = ([a['bbox'][0]-crop_x1, a['bbox'][1]-crop_y1, a['bbox'][2], a['bbox'][3]])
                adj_segmentation = []
                for s in a['segmentation']:
                    adj_segment = []
                    for i in range(0, len(s), 2):
                        s_x, s_y = s[i: i+2].copy()
                        s_x = s_x - crop_x1
                        s_y = s_y - crop_y1
                        adj_segment.append(s_x)
                        adj_segment.append(s_y)
                    adj_segmentation.append(adj_segment)

                b['segmentation'] = adj_segmentation
                b['area'] = a['area']#.copy()
                b['bbox'] = adj_bbox
                b['iscrowd'] = a['iscrowd']#.copy()
                b['id'] = annotation_counter()
                b['image_id'] = image_id
                b['category_id'] = a['category_id']#.copy()

                adj_annotations.append(b)
    
    return adj_annotations


Refer to the following for info about the image orientation attribute

https://gigazine.net/gsc_news/en/20191208-python-exif-orientation/

In [ ]:
# Extract image tiles
import numpy as np
import matplotlib.pyplot as plt
import json

def tile_image(imageInfo, paths, counters, crop_size=[1000, 1000], crop_overlap = 200):
    "                                                                                                  \
    coco_filename = the file name for outout COCO file.                                                \
    imageInfo = the COCO information for the image. See COCO image info details below.                 \
    paths = A dict containing: IMAGE_SRC_PATH, ANNOTATION_PATH, IMAGE_DST_PATH.                        \
    counters = A dict containg: SRC_IMAGE_COUNTER, TILE_IMAGE_COUNTER, ANNOTATION_COUNTER.             \
    crop_size = [width, height] the width and height of the output images.                             \
    crop_overlap = an int value that is used to overlap the tiles.                                     "
    
    img_count = counters['SRC_IMAGE_COUNTER']()
    new_coco_images = []
    new_coco_annotations = []

    # Retrieve the image
    image_path = paths['IMAGE_SRC_PATH']
    with tf.io.gfile.GFile(os.path.join(image_path, '{}'.format(imageInfo['image']['file_name'])), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image_raw = Image.open(encoded_jpg_io)
    image = PIL.ImageOps.exif_transpose(image_raw) #Correct image orientation
    image_size = image.size
    
    for x in range(0, image.size[0]-crop_overlap, crop_size[0]-crop_overlap): #(Start, End, Step)
        if x >= image_size[0]-crop_size[0]: #Adjust for end case
            x = image_size[0]-crop_size[0]
        for y in range(0, image.size[1]-crop_overlap, crop_size[1]-crop_overlap): #(Start, End, Step)
            if y >= image_size[1]-crop_size[1]: #Adjust for end case
                y = image_size[1]-crop_size[1]

            new_image_id = counters['TILE_IMAGE_COUNTER']()

            #Insert the img_count into the file name to create a new file name for each tile
            
            new_coco_image = {}
            new_coco_image = imageInfo['image'].copy()
            new_coco_image['id'] = new_image_id
            new_coco_image['width'], imageInfo['image']['height'] = crop_size
            crop_box = (x, y, x+crop_size[0], y+crop_size[1])

            new_coco_image['file_name'] = filename_append(imageInfo['image']['file_name'], str(new_image_id))
            new_coco_image['flickr_url'] = filename_append(imageInfo['image']['flickr_url'], str(new_image_id))
            new_coco_image['coco_url'] = filename_append(imageInfo['image']['coco_url'], str(new_image_id))
            filename = os.path.join(paths['IMAGE_DST_PATH'], new_coco_image['file_name'])

            #print()
            #print('filename =',)
            #print(filename)
            #print('   source img_count =', img_count, '   new_image_id =', new_image_id)
            #print('   crop box = ', x, y, x+crop_size[0], y+crop_size[1])
            #print('   crop box = ', crop_box)
            
            adj_annotations = []
            adj_annotations = get_bboxes(crop_box, imageInfo, new_image_id, counters['ANNOTATION_COUNTER'])
            #print('   adj_annotations = ', adj_annotations)


            if len(adj_annotations)>0: #If the cropped image contains annotations...
                new_coco_images.append(new_coco_image)
                new_coco_annotations.extend(adj_annotations)

                # save the cropped image to the updated filename - only if the image has annotations
                filename = os.path.join(paths['IMAGE_DST_PATH'], new_coco_image['file_name'])
                image.crop(crop_box).save(filename)
                
                # Display each image and its info
                #t={}
                #t['annotations'] = adj_annotations #Add the annotation info
                #t['image'] = new_coco_image #Add the image info
                #plt_image_with_labels2(image.crop(crop_box), t, categoriesDict)

    #Return generated coco json annotation data
    coco_json_data = {}
    coco_json_data['images'] = new_coco_images
    coco_json_data['annotations'] = new_coco_annotations

    return coco_json_data

In [ ]:
# Display an imageInfo record and the image

# Set counters
counters={'SRC_IMAGE_COUNTER':counter(0), 'TILE_IMAGE_COUNTER':counter(0), 'ANNOTATION_COUNTER':counter(0)}

crop_size = [1000, 1000]
crop_overlap = 200
j = 0

images = []
annotations = []

for image in imageInfoList:
    image_coco_data = tile_image(image, paths, counters, crop_size, crop_overlap)
    images.extend(image_coco_data['images'])
    annotations.extend(image_coco_data['annotations'])
    # Limit the number of images processed
    #j += 1
    #if j > 2:
    #    break

coco_json_data = {}
coco_json_data['info'] = coco_info['info']
coco_json_data['images'] = images
coco_json_data['annotations'] = annotations
coco_json_data['licenses'] = coco_info['licenses']
coco_json_data['categories'] = coco_info['categories']
    
filename = os.path.join(paths['IMAGE_DST_PATH'], files['DST_ANNOTATIONS_FILE'])
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(coco_json_data, f, ensure_ascii=False, indent=4)


In [ ]:
len(coco_info['images'])

In [ ]:
len(coco_info['annotations'])

## COCO image info

[

    {
        "image": {
            "id": 24,
            "width": 6000,
            "height": 1000,
            "file_name": "Day_1_Images_Flight_1_DSC09232_geotag.JPG",
            "license": 0,
            "flickr_url": "./Day_1_Images_Flight_1_DSC09232_geotag.JPG",
            "coco_url": "./Day_1_Images_Flight_1_DSC09232_geotag.JPG",
            "date_captured": ""
            },
        "annotations": [
            {
                "segmentation": [
                    [
                        3631,
                        308,
                        3779,
                        308,
                        3779,
                        446,
                        3631,
                        446
                    ]
                ],
                "area": 20424,
                "bbox": [
                    3631,
                    308,
                    148,
                    138
                ],
                "iscrowd": 0,
                "id": 3,
                "image_id": 24,
                "category_id": 1
            },
            {
                "segmentation": [
                    [
                        3788,
                        529,
                        3896,
                        529,
                        3896,
                        638,
                        3788,
                        638
                    ]
                ],
                "area": 11772,
                "bbox": [
                    3788,
                    529,
                    108,
                    109
                ],
                "iscrowd": 0,
                "id": 4,
                "image_id": 24,
                "category_id": 1
            },
            {
                "segmentation": [
                    [
                        3883,
                        437,
                        4100,
                        437,
                        4100,
                        655,
                        3883,
                        655
                    ]
                ],
                "area": 47306,
                "bbox": [
                    3883,
                    437,
                    217,
                    218
                ],
                "iscrowd": 0,
                "id": 5,
                "image_id": 24,
                "category_id": 1
            }
        ]
    }
]

## COCO JSON structure

{
    
    "info": {
        "year": 2022,
        "version": "1.0",
        "description": "VIA project exported to COCO format using VGG Image Annotator (http://www.robots.ox.ac.uk/~vgg/software/via/)",
        "contributor": "",
        "url": "http://www.robots.ox.ac.uk/~vgg/software/via/",
        "date_created": "Sat Jul 30 2022 19:22:25 GMT+1200 (New Zealand Standard Time)"
    },

    "images": [
        {
            "id": 23,
            "width": 6000,
            "height": 4000,
            "file_name": "Day_1_Images_Flight_1_DSC09231_geotag.JPG",
            "license": 0,
            "flickr_url": "./Day_1_Images_Flight_1_DSC09231_geotag.JPG",
            "coco_url": "./Day_1_Images_Flight_1_DSC09231_geotag.JPG",
            "date_captured": ""
        },
    ],
    
    "annotations": [
        {
            "segmentation": [
                [
                    1006,
                    1384,
                    1084,
                    1384,
                    1084,
                    1596,
                    1006,
                    1596
                ]
            ],
            "area": 16536,
            "bbox": [
                1006,
                1384,
                78,
                212
            ],
            "iscrowd": 0,
            "id": 1,
            "image_id": 1,
            "category_id": 3
        },
    ],
    
    
    "licenses": [
        {
            "id": 0,
            "name": "Unknown License",
            "url": ""
        }
    ],
    
    "categories": [
        {
            "supercategory": "Damage",
            "id": 1,
            "name": "None"
        },
    ]    
}